In [58]:
%pip install torch
%pip install torchvision
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!kaggle datasets download "aryashah2k/breast-ultrasound-images-dataset" -p "./data"
!unzip data/breast-ultrasound-images-dataset.zip
!rm -rf Dataset_BUSI_with_GT/normal
!mv Dataset_BUSI_with_GT/* data
!rm -rf Dataset_BUSI_with_GT
!rm -rf data/breast-ultrasound-images-dataset.zip
!find data -type f -name "*_mask*.png" -delete

!kaggle datasets download "vuppalaadithyasairam/ultrasound-breast-images-for-breast-cancer" -p "./data"
!unzip data/ultrasound-breast-images-for-breast-cancer.zip
!mv "ultrasound breast classification/train/benign"/* data/benign
!mv "ultrasound breast classification/val/benign"/* data/benign
!mv "ultrasound breast classification/train/malignant"/* data/malignant
!mv "ultrasound breast classification/val/malignant"/* data/malignant
!rm -rf "ultrasound breast classification"
!rm -rf data/ultrasound-breast-images-for-breast-cancer.zip

Dataset URL: https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset
License(s): CC0-1.0
  0%|                                                | 0.00/195M [00:00<?, ?B/s]
100%|████████████████████████████████████████| 195M/195M [00:00<00:00, 3.77GB/s]
Archive:  data/breast-ultrasound-images-dataset.zip
  inflating: Dataset_BUSI_with_GT/benign/benign (1).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (1)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102)_ma

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

print(f"Device: {device}")

Device: mps


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = datasets.ImageFolder(root="data", transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [3]:
class BreastCancerCNN(nn.Module):
    def __init__(self):
        super(BreastCancerCNN, self).__init__()

        # Convolutional layers
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.01),
            nn.MaxPool2d(2, 2),
        )
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        # Fully connected and dropout layers
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.5),
            nn.Linear(32, 2),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
!ls data/benign | wc -l
!ls data/malignant | wc -l

In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        cross_entropy_loss = F.cross_entropy(inputs, targets)
        probs = torch.exp(-cross_entropy_loss)
        focal_loss = self.alpha * (1 - probs) ** self.gamma * cross_entropy_loss

        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

In [6]:
cancer_net = BreastCancerCNN()
cancer_net = cancer_net.to(device)
loss_fn = FocalLoss(alpha=0.25, gamma=4)
optimizer = optim.AdamW(cancer_net.parameters(), lr=3e-4, weight_decay=1e-4)
num_epochs = 40

In [7]:
for epoch in range(num_epochs):
    i = 0
    start_time = time.time()
    cancer_net.train()
    current_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cancer_net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        current_loss += loss.item()
        if i % 200 == 199 or i == len(train_loader) - 1:
            end_time = time.time()
            print(f"[Epoch: {epoch + 1}/{num_epochs}, Image: {i + 1}/{len(train_loader)}] Loss: {current_loss:0.5f}, Time Elapsed: {end_time - start_time:0.5f}s")
            current_loss = 0.0
            start_time = end_time
        i += 1

print("Training complete!")

[Epoch: 1/40, Image: 200/7333] Loss: 2.17156, Time Elapsed: 2.91580s
[Epoch: 1/40, Image: 400/7333] Loss: 2.16445, Time Elapsed: 2.34045s
[Epoch: 1/40, Image: 600/7333] Loss: 2.15635, Time Elapsed: 2.29179s
[Epoch: 1/40, Image: 800/7333] Loss: 2.18533, Time Elapsed: 2.30831s
[Epoch: 1/40, Image: 1000/7333] Loss: 2.17760, Time Elapsed: 2.27233s
[Epoch: 1/40, Image: 1200/7333] Loss: 2.17417, Time Elapsed: 2.30281s
[Epoch: 1/40, Image: 1400/7333] Loss: 2.09291, Time Elapsed: 2.44413s
[Epoch: 1/40, Image: 1600/7333] Loss: 2.20120, Time Elapsed: 2.51120s
[Epoch: 1/40, Image: 1800/7333] Loss: 2.17973, Time Elapsed: 2.53749s
[Epoch: 1/40, Image: 2000/7333] Loss: 2.12800, Time Elapsed: 2.16184s
[Epoch: 1/40, Image: 2200/7333] Loss: 2.16795, Time Elapsed: 2.21827s
[Epoch: 1/40, Image: 2400/7333] Loss: 2.09521, Time Elapsed: 2.61051s
[Epoch: 1/40, Image: 2600/7333] Loss: 2.08782, Time Elapsed: 2.42616s
[Epoch: 1/40, Image: 2800/7333] Loss: 2.09769, Time Elapsed: 2.25928s
[Epoch: 1/40, Image: 300

In [8]:
correct_test = 0
total_test = 0

correct_train = 0
total_train = 0

with torch.no_grad():
    cancer_net.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = cancer_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    print(f"Test accuracy: {correct_test}/{total_test} => {100 * correct_test/total_test:0.4f}%")

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = cancer_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    print(f"Train accuracy: {correct_train}/{total_train} => {100 * correct_train/total_train:0.4f}%")

Test accuracy: 1797/1834 => 97.9826%
Train accuracy: 7287/7333 => 99.3727%


In [9]:
torch.save(cancer_net.state_dict(), "breast_cancer_cnn.pth")